Import all the necessary libraries

In [1]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout

Load the dataset

In [2]:
(X_train,Y_train),(X_test,Y_test) = mnist.load_data()

In [3]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


Reshape the dataset into a vector

In [4]:
X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],
                          X_train.shape[2],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],
                        X_test.shape[2],1)

In [5]:
Y_train = to_categorical(Y_train)
Y_test = to_categorical(Y_test)

Normalise the data . Pixels are in the value range of 0-255. We divide each pixel by 255 so that now the pixel value ranges as 0-1.

In [6]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train/= 255.0
X_test/=255.0

Design the neural network

In [7]:
model = Sequential()
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(28,28,1)))
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.2))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(512,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(10,activation='softmax'))

In [8]:
model.compile(optimizer='adam' , 
              loss ='categorical_crossentropy',
              metrics =['accuracy'])

In [9]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        print(logs.get('accuracy'))
        if(logs.get('accuracy')>=0.992):
            print("\nReached 99.2% accuracy so cancelling training!")
            self.model.stop_training = True
        
callbacks = myCallback()

In [ ]:
history = model.fit(X_train,Y_train, epochs = 15, 
                    validation_data=(X_test,Y_test),
                    callbacks = [callbacks])

Epoch 1/15
1875/1875 [==============================] - 74s 39ms/step - loss: 0.3648 - accuracy: 0.8795 - val_loss: 0.0506 - val_accuracy: 0.9824
0.9466000199317932
Epoch 2/15
1875/1875 [==============================] - 72s 38ms/step - loss: 0.0714 - accuracy: 0.9787 - val_loss: 0.0390 - val_accuracy: 0.9878
0.9799333214759827
Epoch 3/15
1496/1875 [======================>.......] - ETA: 13s - loss: 0.0490 - accuracy: 0.9844

Predict on the image. the threshhold values depends on the input picture.

In [11]:
def prepImg(data):
    return cv2.resize(data,(28,28)).reshape(28,28,1)/255.0

img = cv2.imread("photo_new.jpg")
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray, (5,5) , 0)
ret , im_th = cv2.threshold(blur, 180 , 400 , cv2.THRESH_BINARY_INV)
ctrs , hier = cv2.findContours(im_th.copy(),cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
rects = [cv2.boundingRect(ctr) for ctr in ctrs]

cv2.imshow('Threshhold',im_th)
cv2.waitKey(0)

-1

In [12]:
from tensorflow.keras.preprocessing.image import img_to_array
for x,y,w,h in rects :
    
    if y>=3:
        y-=3
    else :
        y=0
    if x>=3:
        x-=3
    else:
        x=0
    w+=3
    h+=3
    cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
    sliced = im_th[y:y+h,x:x+w]
    sliced = img_to_array(sliced,dtype='float32')
    sliced = prepImg(sliced)
    sliced = np.expand_dims(sliced , axis = 0)
    prediction = model.predict_classes(sliced)
    cv2.putText(img, str(prediction[0]), (x+w,y+int(h/2)), cv2.FONT_HERSHEY_SIMPLEX ,  1, (0,255,0) , 2, cv2.LINE_AA) 
cv2.imshow("IMAGE",img)
cv2.waitKey(0)
cv2.destroyAllWindows()